# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [57]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import holoviews as hv

from bokeh.models import HoverTool

In [58]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [59]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [60]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [61]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [62]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [63]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11

k = list(range(1,12))

In [64]:
# Create an empy list to store the inertia values

inertia = []

In [65]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [66]:
# Create a dictionary with the data to plot the Elbow curve

elbow_dict = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow = pd.DataFrame(elbow_dict)



In [67]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.


#Displays elbow line graph 
#df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


plot_title='Inertia vs. Number of Clusters (k) in KMeans Algorithm'
elbow_plot_1 = df_elbow.hvplot(title = plot_title, x='k', y='inertia', xticks=k)


#elbow_plot_1


try:
    plot_filename='Elbow_Plot_1.png'
    hvplot.save(elbow_plot_1, 'Images/'+plot_filename)
except Exception as e:
    print(type(e), e)

elbow_plot_1



<class 'RuntimeError'> To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')


:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** Judging by the graph, the line's slope––which displays change in inertia–– trends to near 0 when k > 4. Therefore, the best number of clusters is k = 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [68]:
# Initialize the K-Means model using the best value for k

model = KMeans(n_clusters=4, random_state=0)

In [69]:
# Fit the K-Means model using the scaled data

model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=0)

In [70]:
# Predict the clusters to group the cryptocurrencies using the scaled data

clusters_predictions = model.predict(df_market_data_scaled)


# View the resulting array of cluster values.


display(clusters_predictions)

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 0, 3, 3, 2, 3, 3, 3, 3],
      dtype=int32)

In [71]:
# Create a copy of the DataFrame

df_clusters = pd.concat([df_elbow, df_market_data_scaled], axis=1, sort=False)

#dfCopy = clusters_predictions.copy()

In [72]:
# Add a new column to the DataFrame with the predicted clusters

#df_market_data_scaled['mkt_cluster'] = clusters
#df_market
# Display sample data
#clusters

# display(df_market_data_scaled)
#display(df_market_data_scaled.head())



# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled['pca_cluster'] = clusters_predictions


# Display sample data
display(df_market_data_scaled.head())


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,pca_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,3
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [73]:
hover_fmt = HoverTool(
    tooltips=[("Coin ID",  "@coin_id"),
              ("Price Change 24h", "$x{0.000}%"),
              ("Price Change 7d", "$y{0.000}%"),
             ],
    formatters = {
#         '@date': 'datetime'
    }
)

In [77]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 

cluster_plot_1 = df_market_data_scaled.hvplot.scatter(
        title = "7 Day vs. 24 Hour Price Change (Clustered)",
        x = "price_change_percentage_24h",
        y = "price_change_percentage_7d",
        

# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

        xlabel = "24 Hour Price Change (%)",
        ylabel = "7 Day Price Change (%)",
        #by = "mkt_cluster",
        #hover_cols = "coin_id",
        #tools = [hover_fmt],
    
)

try:
    plot_filename='Cluster_Plot_1.png'
    hvplot.save(cluster_plot_1, 'Images/'+plot_filename)
except Exception as e:
    print(type(e), e)

cluster_plot_1


<class 'RuntimeError'> To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [104]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3, random_state=0)

In [110]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
df_market_pca_data_5_row = pd.DataFrame(market_pca_data, columns = ["PC1", "PC2", "PC3"] )
df_market_pca_data_5_row.head()

,PC1,PC2,PC3
0,2.034093,-0.641571,-0.357629
1,1.994502,-0.490549,-0.912250
2,-1.068544,-0.417205,0.596311
3,-1.040919,-0.455234,0.467801
4,3.302764,-1.242458,-0.217276


In [107]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

explained_var = pca.explained_variance_ratio_
print(f"{explained_var}")

[0.36941241 0.29169375 0.22893878]


#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** Total explained variance of 3 principal components is: 

In [108]:
print(f"Total explained variance: {explained_var.sum():.2f}%")

Total explained variance: 0.89%


In [115]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data

df_market_pca_data = pd.DataFrame(market_pca_data, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data

df_market_pca_data["coinid"] = df_market_data.index

# Set the coinid column as index

df_market_pca_data.set_index("coinid", inplace=True)

# Display sample data

#display(df_market_pca_data.head())
df_market_pca_data.head()

,PC1,PC2,PC3
coinid,,,
bitcoin,2.034093,-0.641571,-0.357629
ethereum,1.994502,-0.490549,-0.912250
tether,-1.068544,-0.417205,0.596311
ripple,-1.040919,-0.455234,0.467801
bitcoin-cash,3.302764,-1.242458,-0.217276


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [117]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11

k = list(range(1,12))

#display(k)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [118]:
# Create an empy list to store the inertia values

inertia = []

In [120]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list


for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_pca_data)
    inertia.append(model.inertia_)

In [123]:
# Create a dictionary with the data to plot the Elbow curve

elbow_dictionary = {"k":k, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow = pd.DataFrame(elbow_dictionary)

In [127]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_plot_2 = df_elbow.hvplot(title="Inertia vs. Number of Clusters (k)", x="k", y="inertia", xticks=k)



#elbow_plot_2

try:
    plot_filename='Elbow_Plot_2.png'
    hvplot.save(elbow_plot_2, 'Images/'+plot_filename)
except Exception as e:
    print(type(e), e)
    
elbow_plot_2

<class 'RuntimeError'> To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')


:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Judging by the graph, there is a minimal change in inertia for values of k>4. Therefore, the best number of clusters is k=4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Since there is no difference between the original data and the PCA data, the ideal k value for the original dataset is also k=4.

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [128]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [129]:
# Fit the K-Means model using the PCA data
model.fit(df_market_pca_data)

KMeans(n_clusters=4, random_state=0)

In [132]:
# Predict the clusters to group the cryptocurrencies using the PCA data

clusters_predictions = model.predict(df_market_pca_data)

# View the resulting array of cluster values.

display(clusters_predictions)

array([0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 3, 0, 2, 2, 1, 2, 2, 2, 2],
      dtype=int32)

In [137]:
# Create a copy of the DataFrame with the PCA data
df_clusters_predictions = clusters_predictions.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled["pca_cluster"] = df_clusters_predictions
df_market_pca_data["pca_cluster"] = df_clusters_predictions

# Display sample data

#display(df_market_data_scaled.head())
display(df_market_pca_data.head())

,PC1,PC2,PC3,pca_cluster
coinid,,,,
bitcoin,2.034093,-0.641571,-0.357629,0
ethereum,1.994502,-0.490549,-0.912250,0
tether,-1.068544,-0.417205,0.596311,2
ripple,-1.040919,-0.455234,0.467801,2
bitcoin-cash,3.302764,-1.242458,-0.217276,0


In [140]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

cluster_plot_2 = df_market_pca_data.hvplot.scatter(x="PC1", y="PC2", by="pca_cluster", hover_cols=["coinid"], group_label="PCA Cluster",)


try:
    plot_filename="Cluster_Plot_2.png"
    hvplot.save(cluster_plot_2, "Images/"+plot_filename)
except Exception as e:
    print(type(e), e)
    
cluster_plot_2

<class 'RuntimeError'> To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')


:NdOverlay   [pca_cluster]
   :Scatter   [PC1]   (PC2,coinid)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [142]:
# Composite plot to contrast the Elbow curves

sub_height = 400
sub_width = 500

elbow_plot_composite = elbow_plot_1.opts(height=sub_height, width=sub_width,) + elbow_plot_2.opts(height=sub_height, width=sub_width,)

elbow_plot_composite

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [147]:
# Compoosite plot to contrast the clusters

cluster_plot_composite = cluster_plot_1.opts(height=sub_height, width=sub_width,) + cluster_plot_2.opts(height=sub_height, width=sub_width,)

cluster_plot_composite

:Layout
   .Scatter.I   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.I :NdOverlay   [pca_cluster]
      :Scatter   [PC1]   (PC2,coinid)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** # By using Principal Component Analysis to reduce the number of features in the dataset results in a scatter plot showing the datapoints more tightly grouped by clusters than the graph including all the features in the original dataset. 

In [148]:
cluster_plot_3 = df_market_data_scaled.hvplot.scatter(title="7 Day vs. 24 Hour Price Change (Clustered)", x="price_change_percentage_24h", y="price_change_percentage_7d", xlabel="24 Hour Price Change (%)", ylabel="7 Day Price Change (%)", by="pca_cluster", hover_cols="coin_id", tools=[hover_fmt],)

try:
    plot_filename="Cluster_Plot_3.png"
    hvplot.save(cluster_plot_3, 'Images/'+plot_filename)
except Exception as e:
    print(type(e), e)

cluster_plot_3


<class 'RuntimeError'> To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')


:NdOverlay   [pca_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [149]:
cluster_plot_composite_2 = cluster_plot_1.opts(
    height=sub_height,
    width=sub_width,
) + cluster_plot_3.opts(
    height=sub_height,
    width=sub_width,
)

cluster_plot_composite_2

:Layout
   .Scatter.I   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.I :NdOverlay   [pca_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [151]:
cluster_pca_subplots = df_market_pca_data.hvplot.scatter(
    x='PC1',
    y=['PC2', 'PC3'],
#     by='pca_cluster',
    color='pca_cluster',
    hover_cols=['coinid'],
    group_label='PCA Cluster',
    subplots=True,
#     legend=False,
)

try:
    plot_filename='Cluster_PCA_Subplots.png'
    hvplot.save(cluster_pca_subplots, 'Images/'+plot_filename)
except Exception as e:
    print(type(e), e)

# display(df_market_pca_data.head())
display(df_market_pca_data)
cluster_pca_subplots

<class 'RuntimeError'> To use bokeh.io image export functions you need selenium ('conda install selenium' or 'pip install selenium')


,PC1,PC2,PC3,pca_cluster
coinid,,,,
bitcoin,2.034093,-0.641571,-0.357629,0
ethereum,1.994502,-0.490549,-0.912250,0
tether,-1.068544,-0.417205,0.596311,2
ripple,-1.040919,-0.455234,0.467801,2
bitcoin-cash,3.302764,-1.242458,-0.217276,0
binancecoin,2.505692,-0.575832,-0.129389,0
chainlink,2.770042,-0.495691,-1.890371,0
cardano,2.372449,-0.389146,-0.992513,0
litecoin,1.832344,-0.678797,-0.741581,0


:NdLayout   [PCA Cluster]
   :Scatter   [PC1]   (value,pca_cluster,coinid)